In [16]:
import tensorflow as tf
import glob
#import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from IPython import display
from PIL import Image
import cv2 as cv

# Use to mount Gdrive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
!cp /mydrive/images.zip ../

In [ ]:
!unzip ../images.zip -d data/

# Load images from folder and to a list 

In [17]:
filelist="./images/*.png"

In [22]:
import glob
all_images = []
for index, filename in enumerate(glob.glob(filelist)):
    image = cv.imread(filename)
    all_images.append(image)

X = np.array(all_images)

In [29]:
X

array([[[[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [ 42,  31,  56],
         [ 33,  28,  48],
         [ 29,  31,  35]],

        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [ 44,  46,  67],
         [ 31,  32,  44],
         [ 27,  24,  40]],

        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [ 70,  71,  97],
         [ 30,  30,  44],
         [ 27,  24,  40]],

        ...,

        [[129,  92,  87],
         [125, 100,  97],
         [241, 244, 243],
         ...,
         [160, 129, 134],
         [237, 222, 226],
         [255, 251, 253]],

        [[132,  95,  92],
         [111,  78,  75],
         [195, 191, 189],
         ...,
         [220, 193, 202],
         [247, 243, 244],
         [253, 254, 254]],

        [[112,  85,  83],
         [128,  95,  92],
         [119,  98,  96],
         ...,
         [233, 217, 224],
        

In [27]:
train_images = X.reshape(X.shape[0], 64 , 64 , 3).astype('float32')
train_images = (train_images - 127.5) / 127.5 # Normalize the images to [-1, 1]

In [ ]:
BUFFER_SIZE =1000
BATCH_SIZE = 128
#batch and shuffle the data
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
inputshape=10

# Generator Model

In [ ]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(8*8*256, use_bias=False, input_shape=(inputshape,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((8, 8, 256)))

    model.add(layers.Conv2DTranspose(256, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))

    return model

In [ ]:
make_generator_model().summary()

In [ ]:
#noise = tf.random.normal([1, 100])
z = np.random.uniform(-1., 1., size=[1, inputshape])
#noise
z

In [ ]:
generator = make_generator_model()
generated_image = generator(z, training=False)

plt.imshow(generated_image[0])

# Discriminatior Model

In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                     input_shape=[64, 64, 3]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

In [ ]:
discriminator = make_discriminator_model()
decision = discriminator(generated_image)
print (decision)

## Loss Function

In [ ]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [ ]:
EPOCHS = 10000
noise_dim = inputshape
num_examples_to_generate = 1

# We will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [ ]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images):
    #noise = tf.random.normal([BATCH_SIZE, noise_dim])
    noise = np.random.uniform(-1., 1., size=[BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)
    
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()
        
        for a in dataset:
            train_step(a)

        # Produce images for the GIF as we go
        display.clear_output(wait=True)
        generate_and_save_images(generator,
                                 epoch + 1,
                                 seed)

        # Save the model every 200 epochs
        if (epoch + 1) % 200 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)
        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

    #Generate after the final epoch
    display.clear_output(wait=True)
    generate_and_save_images(generator,
                           epochs,
                           seed)

In [ ]:
def generate_and_save_images(model, epoch, test_input):
    # Notice `training` is set to False.
    # This is so all layers run in inference mode (batchnorm).
    predictions = model(test_input, training=False)

    fig = plt.figure(figsize=(8,8))
    #plt.imshow(predictions[0])
    plt.imshow(predictions[0])
    #for i in range(predictions.shape[0]):
    #    plt.subplot(4, 4, i+1)
     #   plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5)
    #    plt.axis('off')
    pa="./sav/"
    na=epoch
    ex=".png"
    plt.savefig(pa+str(epoch)+ex)
    plt.show()

In [ ]:
train(train_dataset, EPOCHS)

In [ ]:
z1=np.random.uniform(-1., 1., size=[1, inputshape])
predictions = generator(z1, training=False)
fig = plt.figure(figsize=(8,8))
plt.imshow(predictions[0, :, :, 0])

In [ ]:
plt.imshow(predictions[0, :, :, 0] * 127.5 + 127.5, cmap='gray')

In [ ]:
fig = plt.figure(figsize=(8,8))
plt.imshow(predictions[0])

# Prediction with random seed

In [ ]:
for i in range (0,5):
    z1=np.random.uniform(-1., 1., size=[1, inputshape])
    predictions = generator(z1, training=False)
    fig = plt.figure(figsize=(4,4))
    plt.imshow(predictions[0])
    """pa="./aa/"
    na=i
    ex=".png"
    plt.savefig(pa+str(na)+ex)"""

# Restore checkpoint

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))